In [2]:
library(magrittr)
library(data.table)
library(ggrepel)
library(ggpubr)
library(vroom)
library(scales)
library(scico)
library(ggridges)
library(knitr)
library(fs)
library(patchwork)
library(ggnewscale)
library(tidyverse)

source("../src/dms-analysis-utils.R")

colors <- c("Neither" = "gray",
            "Both" = "purple",
            "BMS-986202" = "red",
            "Zasocitinib" = "blue")

aa_order <- c("G","A","P","K","H",
              "R","T","S","C","E",
              "D","M","V","I","L",
              "Y","W","F","Q","N")

In [14]:
sumstats_resist <- read_tsv("../sumstats/TYK2/run4/tyk2-assay4.tsv") %>%
    filter(condition %in% c("IFNalpha+BMS-986202_1e-06 - None_0", "IFNalpha+Zasocitinib_1e-06 - None_0"),
           aa != "*") %>%
    select(pos, drug, aa, statistic, p.adj) %>%
    pivot_wider(names_from = drug, values_from = c(statistic, p.adj)) %>%
    mutate(`FDR < 0.01` = case_when(`p.adj_IFNalpha+BMS-986202` < 0.01 & `p.adj_IFNalpha+Zasocitinib` < 0.01 ~ "Both",
                           `p.adj_IFNalpha+BMS-986202` < 0.01 & `p.adj_IFNalpha+Zasocitinib` > 0.01 ~ "BMS-986202",
                           `p.adj_IFNalpha+BMS-986202` > 0.01 & `p.adj_IFNalpha+Zasocitinib` < 0.01 ~ "Zasocitinib",
                           TRUE ~ "Neither"),
          `FDR < 0.01` = if_else(`statistic_IFNalpha+Zasocitinib` < 0 & `statistic_IFNalpha+BMS-986202` < 0,
                                 "Neither",
                                 `FDR < 0.01`),
          aa = factor(aa, levels = aa_order))

sig_vars_resist <- sumstats_resist %>% filter(`FDR < 0.01` != "Neither") %>% pull(pos) %>% unique()
prepped_data_resist <- sumstats_resist %>% filter(pos %in% sig_vars_resist)

Rows: 118600 Columns: 14
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr (6): drug, clone, chunk, aa, version, condition
dbl (8): conc, pos, estimate, std.error, dispersion, statistic, p.value, p.adj

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [16]:
sumstats_potentiate <- read_tsv("../sumstats/TYK2/run7/tyk2-assay7-unnormalized.tsv")
bms_poten <- compute_difference(test = "IFNalpha100+BMS-986202_2e-08",
                                control = "IFNalpha100_0",
                                sumstats_potentiate) 

zaso_poten <- compute_difference(test = "IFNalpha100+Zasocitinib_7e-09",
                                control = "IFNalpha100_0",
                                sumstats_potentiate) 

sumstats_potentiate <- bind_rows(bms_poten, zaso_poten) %>%
    filter(aa != "*") %>%
    mutate(statistic = estimate / std.error,
           p.value = (1 - pnorm(abs(statistic))) * 2,,
           p.adj = p.adjust(p.value, method = "BH"),
           drug = gsub("_.*", "", condition)) %>%
    select(pos, drug, aa, statistic, p.adj) %>%
    pivot_wider(names_from = drug, values_from = c(statistic, p.adj)) %>%
    mutate("FDR < 0.01" = case_when(`p.adj_IFNalpha100+BMS-986202` < 0.01 & `p.adj_IFNalpha100+Zasocitinib` > 0.01 ~ "BMS-986202",
                               `p.adj_IFNalpha100+BMS-986202` > 0.01 & `p.adj_IFNalpha100+Zasocitinib` < 0.01 ~ "Zasocitinib",
                               `p.adj_IFNalpha100+BMS-986202` < 0.01 & `p.adj_IFNalpha100+Zasocitinib` < 0.01 ~ "Both",
                               TRUE ~ "Neither"),
           `FDR < 0.01` = if_else(`statistic_IFNalpha100+BMS-986202` > 0,
                                 "Neither",
                                 `FDR < 0.01`),
           aa = factor(aa, levels = aa_order))

sig_vars_potentiate <- sumstats_potentiate %>% filter(`FDR < 0.01` != "Neither") %>% pull(pos) %>% unique()
prepped_data_potentiate <- sumstats_potentiate %>% filter(pos %in% sig_vars_potentiate)
names(prepped_data_potentiate) <- names(prepped_data_resist)

Rows: 118595 Columns: 14
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr (6): clone, chunk, condition, aa, version, drug
dbl (8): pos, estimate, std.error, statistic, p.value, dispersion, conc, p.adj

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [37]:
colors <- c("Neither" = "white",
            "Both" = "purple",
            "BMS-986202" = "red",
            "Zasocitinib" = "blue")

plot_white <- bind_rows(
    prepped_data_potentiate %>% mutate("effect" = "potentiation"),
    prepped_data_resist %>% mutate("effect" = "resistance")
) %>% ggplot(aes(text = str_c(pos, aa))) +
    geom_tile(aes(x = as.factor(pos),
                  y = fct_rev(aa),
                  fill = `FDR < 0.01`)) +
    scale_fill_manual(values = colors) +
    xlab("Position") + ylab("") +
    theme_pubr() +
    facet_grid(cols = vars(effect), scale = "free_x", space = "free")

colors <- c("Neither" = "gray",
            "Both" = "purple",
            "BMS-986202" = "red",
            "Zasocitinib" = "blue")

plot_gray <- bind_rows(
    prepped_data_potentiate %>% mutate("effect" = "potentiation"),
    prepped_data_resist %>% mutate("effect" = "resistance")
) %>% ggplot(aes(text = str_c(pos, aa))) +
    geom_tile(aes(x = as.factor(pos),
                  y = fct_rev(aa),
                  fill = `FDR < 0.01`)) +
    scale_fill_manual(values = colors) +
    xlab("Position") + ylab("") +
    theme_pubr() +
    facet_grid(cols = vars(effect), scale = "free", space = "free")

In [38]:
pdf("../TYK2-chem-footprint.pdf", width = 15, height = 7)
plot_white
plot_gray
dev.off()

png 
  2